In [73]:
import pandas as pd
import numpy as np

data = pd.read_csv("salary-train.csv", header = 0)
data.head()
test = pd.read_csv("salary-test-mini.csv", header = 0)
#test.head()

In [74]:
y = data['SalaryNormalized']
#предобработка
df = pd.DataFrame(data)
df = df.loc[:,:'ContractTime']
# к нижнему регистру
df['FullDescription'] = df['FullDescription'].str.lower()
df['LocationNormalized'] = df['LocationNormalized'].str.lower()
#Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.
df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [75]:
df.fillna('nan', inplace=True)
df

,FullDescription,LocationNormalized,ContractTime
0,international sales manager london k ...,london,permanent
1,an ideal opportunity for an individual that ha...,london,permanent
2,online content and brand manager luxury reta...,south east london,permanent
3,a great local marketleader is seeking a perman...,dereham,permanent
4,registered nurse rgn nursing home for young...,sutton coldfield,nan
...,...,...,...
59995,as a result of continued growth first class s...,whitley bay,contract
59996,php mvc web developer macclesfieldcirca ...,macclesfield,permanent
59997,staff nurse nursing home baldock white recru...,baldock,nan
59998,this is one of the best agency side opportunit...,the city,permanent


In [76]:
# предобработка теста
df1 = pd.DataFrame(test)
df1 = df1.loc[:,:'ContractTime']
# к нижнему регистру
df1['FullDescription'] = df1['FullDescription'].str.lower()
df1['LocationNormalized'] = df1['LocationNormalized'].str.lower()
#Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.
df1['FullDescription'] = df1['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
df1.fillna('nan', inplace=True)
df1

,FullDescription,LocationNormalized,ContractTime
0,we currently have a vacancy for an hr project ...,milton keynes,contract
1,a web developer opportunity has arisen with an...,manchester,permanent


In [77]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer=TfidfVectorizer(min_df=5)

X_train_vec = vectorizer.fit_transform(df['FullDescription'])

X_test_vec = vectorizer.transform(df1['FullDescription'])

In [78]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(df1[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [88]:
import scipy

X_for_train = scipy.sparse.hstack([X_train_vec, X_train_categ])
X_for_test = scipy.sparse.hstack([X_test_vec, X_test_categ])

In [91]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0, random_state = 241)
clf.fit(X_for_train, y)
pred = clf.predict(X_for_test)

print (' '.join(map(lambda x: str(round(x,2)), pred)))

56563.5 37141.0
